In [1]:
import sys
sys.path.append("C:/Users/user/meepc")
import numpy as np
import pandas as pd
from models import Hankel,Corrhankel,Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def calculate_fpr(y_actual, y_prediction):
    fp = 0
    tn = 0
    
    for i in range(len(y_actual)):
        if y_actual[i] == 0 and y_prediction[i] == 1:
            fp += 1
        elif y_actual[i] == 0 and y_prediction[i] == 0:
            tn += 1
    fpr= fp/(fp+tn)
    return fpr

In [3]:
df1 = pd.read_csv('~/data/ctown/dataset03.csv')
df2 = pd.read_csv('~/data/ctown/dataset04.csv')

train_normal = pd.concat((df1,df2[df2['ATT_FLAG']==0]),axis=0,ignore_index=True)
train_attack = df2[df2['ATT_FLAG']==1]

In [4]:
sensors = [col for col in train_normal.columns if col not in ['DATETIME','ATT_FLAG']]

In [5]:
scaler = StandardScaler()
X_normal = pd.DataFrame(index=train_normal.index, columns=sensors, data=scaler.fit_transform(train_normal[sensors]))
X_attack = train_attack[sensors].reset_index().drop(columns=['index'])

In [6]:
hankel = Hankel()
corrhankel = Corrhankel()
lag = 60
stride = 0.5

In [7]:
corr_normal,nolag_normal = corrhankel.fit(X_normal.to_numpy(),lag,stride)

In [8]:
corr_attack,nolag_attack = corrhankel.fit(X_attack.to_numpy(),lag,stride)

In [9]:
df_test = pd.read_csv('~/data/ctown/test_dataset.csv')

# Epasad with 1 cluster and no threshold tuning (training attack included in test data)

In [10]:
test_combined = pd.concat((df_test,train_attack),axis=0)
X_test = pd.DataFrame(index=test_combined.index, columns=sensors, data=scaler.fit_transform(test_combined[sensors]))
corr_test,nolag_test = corrhankel.fit(X_test.to_numpy(),lag,stride)
Y_test = test_combined.loc[:,'ATT_FLAG']

In [11]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [12]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
fpr = []
for i,sens in enumerate(sensors):
    train_normal = X_normal.loc[:,sens].values
    train_attack = X_attack.loc[:,sens].values
    model = Pipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,only_corr=True)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test,corr_test[:,i].reshape(nolag_test,len(X_test.columns)).T)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))
    fpr.append(calculate_fpr(y_actual,y_predicted))
    sensor_models.append(model)    

In [13]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [14]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))
print("False Positive Rate ",calculate_fpr(y_actual,y_predicted))

Accuracy  0.8
Precision  0.7735849056603774
Recall  0.8913043478260869
F1-score  0.8282828282828283
False Positive Rate  0.3076923076923077


In [15]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))
print("fpr -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fpr).mean(), np.median(np.asarray(fpr)),np.asarray(fpr).min(), np.asarray(fpr).max()))

Accuracy -  Mean: 0.5398084815321479 Median : 0.5294117647058824 Min : 0.4235294117647059 Max : 0.6588235294117647
precision - Mean: 0.752018441594725 Median : 0.9 Min : 0.0 Max : 1.0
recall -    Mean: 0.16632962588473205 Median : 0.15217391304347827 Min : 0.0 Max : 0.3695652173913043
f1 -        Mean: 0.2641570057445195 Median : 0.2545454545454546 Min : 0.0 Max : 0.5396825396825397
fpr -        Mean: 0.01967799642218247 Median : 0.0 Min : 0.0 Max : 0.10256410256410256


# Multiple clusters + No threshold tuning (training attack mixed in test data)

In [16]:
cluster_accuracy_1 = []
cluster_precision_1 = []
cluster_recall_1 = []
cluster_fscore_1 = []
cluster_fpr_1 = []
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    for i,sens in enumerate(sensors):
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k = k,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,only_corr=True)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_test.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual,y_predicted))
        sensor_models.append(model)  
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int) 
    cluster_accuracy_1.append(accuracy_score(y_actual, y_predicted))
    cluster_precision_1.append(precision_score(y_actual,y_predicted))
    cluster_recall_1.append(recall_score(y_actual,y_predicted))
    cluster_fscore_1.append(f1_score(y_actual,y_predicted))
    cluster_fpr_1.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy_1[-1])
    print("Precision ",cluster_precision_1[-1])
    print("Recall ",cluster_recall_1[-1])
    print("F1-score ",cluster_fscore_1[-1])
    print("False Positive Rate ",cluster_fpr_1[-1],"/n") 

------Number of Clusters:  2 -----------
Accuracy  0.7411764705882353
Precision  0.6818181818181818
Recall  0.9782608695652174
F1-score  0.8035714285714285
False Positive Rate  0.5384615384615384 /n
------Number of Clusters:  3 -----------
Accuracy  0.6705882352941176
Precision  0.6216216216216216
Recall  1.0
F1-score  0.7666666666666667
False Positive Rate  0.717948717948718 /n
------Number of Clusters:  4 -----------
Accuracy  0.6235294117647059
Precision  0.5897435897435898
Recall  1.0
F1-score  0.7419354838709677
False Positive Rate  0.8205128205128205 /n
------Number of Clusters:  5 -----------
Accuracy  0.5882352941176471
Precision  0.569620253164557
Recall  0.9782608695652174
F1-score  0.7200000000000001
False Positive Rate  0.8717948717948718 /n
------Number of Clusters:  6 -----------
Accuracy  0.6
Precision  0.575
Recall  1.0
F1-score  0.7301587301587301
False Positive Rate  0.8717948717948718 /n
------Number of Clusters:  7 -----------
Accuracy  0.5882352941176471
Precision 

In [17]:
print("Accuracy Scores: ",cluster_accuracy_1)
print("Precision Scores: ",cluster_precision_1)
print("Recall Scores: ",cluster_recall_1)
print("F1 Scores: ",cluster_fscore_1)
print("False Positive Rates: ",cluster_fpr_1)

Accuracy Scores:  [0.7411764705882353, 0.6705882352941176, 0.6235294117647059, 0.5882352941176471, 0.6, 0.5882352941176471]
Precision Scores:  [0.6818181818181818, 0.6216216216216216, 0.5897435897435898, 0.569620253164557, 0.575, 0.5679012345679012]
Recall Scores:  [0.9782608695652174, 1.0, 1.0, 0.9782608695652174, 1.0, 1.0]
F1 Scores:  [0.8035714285714285, 0.7666666666666667, 0.7419354838709677, 0.7200000000000001, 0.7301587301587301, 0.7244094488188977]
False Positive Rates:  [0.5384615384615384, 0.717948717948718, 0.8205128205128205, 0.8717948717948718, 0.8717948717948718, 0.8974358974358975]


# Multiple clusters + No threshold tuning (No concat of training and test data)

In [18]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
Y_test = df_test.loc[:,'ATT_FLAG']
corr_test,nolag_test = corrhankel.fit(X_test.to_numpy(),lag,stride)

In [19]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [20]:
cluster_accuracy_2 = []
cluster_precision_2 = []
cluster_recall_2 = []
cluster_fscore_2 = []
cluster_fpr_2 = []
for k in range(2,8):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    for sens in sensors:
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=1,tune=False,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,corr_attack=corr_attack[:,i].reshape(nolag_attack,len(X_attack.columns)).T,only_corr=True)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test[:,i].reshape(nolag_test,len(X_test.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    cluster_accuracy_2.append(accuracy_score(y_actual, y_predicted))
    cluster_precision_2.append(precision_score(y_actual,y_predicted))
    cluster_recall_2.append(recall_score(y_actual,y_predicted))
    cluster_fscore_2.append(f1_score(y_actual,y_predicted))
    cluster_fpr_2.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy_2[-1])
    print("Precision ",cluster_precision_2[-1])
    print("Recall ",cluster_recall_2[-1])
    print("F1-score ",cluster_fscore_2[-1])
    print("False Positive Rate ",cluster_fpr_2[-1],"/n") 

------Number of Clusters:  2 -----------
Accuracy  0.8088235294117647
Precision  0.8076923076923077
Recall  0.7241379310344828
F1-score  0.7636363636363636
False Positive Rate  0.1282051282051282 /n
------Number of Clusters:  3 -----------
Accuracy  0.8088235294117647
Precision  0.8076923076923077
Recall  0.7241379310344828
F1-score  0.7636363636363636
False Positive Rate  0.1282051282051282 /n
------Number of Clusters:  4 -----------
Accuracy  0.8088235294117647
Precision  0.8076923076923077
Recall  0.7241379310344828
F1-score  0.7636363636363636
False Positive Rate  0.1282051282051282 /n
------Number of Clusters:  5 -----------
Accuracy  0.8088235294117647
Precision  0.8076923076923077
Recall  0.7241379310344828
F1-score  0.7636363636363636
False Positive Rate  0.1282051282051282 /n
------Number of Clusters:  6 -----------
Accuracy  0.8088235294117647
Precision  0.8076923076923077
Recall  0.7241379310344828
F1-score  0.7636363636363636
False Positive Rate  0.1282051282051282 /n
-----

In [21]:
print("Accuracy Scores: ",cluster_accuracy_2)
print("Precision Scores: ",cluster_precision_2)
print("Recall Scores: ",cluster_recall_2)
print("F1 Scores: ",cluster_fscore_2)
print("False Positive Rates: ",cluster_fpr_2)

Accuracy Scores:  [0.8088235294117647, 0.8088235294117647, 0.8088235294117647, 0.8088235294117647, 0.8088235294117647, 0.7941176470588235]
Precision Scores:  [0.8076923076923077, 0.8076923076923077, 0.8076923076923077, 0.8076923076923077, 0.8076923076923077, 0.8]
Recall Scores:  [0.7241379310344828, 0.7241379310344828, 0.7241379310344828, 0.7241379310344828, 0.7241379310344828, 0.6896551724137931]
F1 Scores:  [0.7636363636363636, 0.7636363636363636, 0.7636363636363636, 0.7636363636363636, 0.7636363636363636, 0.7407407407407408]
False Positive Rates:  [0.1282051282051282, 0.1282051282051282, 0.1282051282051282, 0.1282051282051282, 0.1282051282051282, 0.1282051282051282]


# Multiple clusters + Threshold tuning (No concat of training and test data)

In [22]:
cluster_accuracy_3 = []
cluster_precision_3 = []
cluster_recall_3 = []
cluster_fscore_3 = []
cluster_fpr_3 = []
for k in range(2,10):
    sensor_models = []
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    fpr = []
    for sens in sensors:
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = Pipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k = k,tune=True,kscore_init='inertia',corr_normal=corr_normal[:,i].reshape(nolag_normal,len(X_normal.columns)).T,corr_attack=corr_attack[:,i].reshape(nolag_attack,len(X_attack.columns)).T,only_corr=True)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test,corr_test=corr_test[:,i].reshape(nolag_test,len(X_test.columns)).T)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted))
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))
        fpr.append(calculate_fpr(y_actual,y_predicted))
        sensor_models.append(model)    
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    cluster_accuracy_3.append(accuracy_score(y_actual, y_predicted))
    cluster_precision_3.append(precision_score(y_actual,y_predicted))
    cluster_recall_3.append(recall_score(y_actual,y_predicted))
    cluster_fscore_3.append(f1_score(y_actual,y_predicted))
    cluster_fpr_3.append(calculate_fpr(y_actual,y_predicted))
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",cluster_accuracy_3[-1])
    print("Precision ",cluster_precision_3[-1])
    print("Recall ",cluster_recall_3[-1])
    print("F1-score ",cluster_fscore_3[-1])
    print("False Positive Rate ",cluster_fpr_3[-1],"/n") 

------Number of Clusters:  2 -----------
Accuracy  0.6617647058823529
Precision  0.575
Recall  0.7931034482758621
F1-score  0.6666666666666666
False Positive Rate  0.4358974358974359 /n
------Number of Clusters:  3 -----------
Accuracy  0.7205882352941176
Precision  0.6190476190476191
Recall  0.896551724137931
F1-score  0.732394366197183
False Positive Rate  0.41025641025641024 /n
------Number of Clusters:  4 -----------
Accuracy  0.7058823529411765
Precision  0.6
Recall  0.9310344827586207
F1-score  0.7297297297297297
False Positive Rate  0.46153846153846156 /n
------Number of Clusters:  5 -----------
Accuracy  0.6176470588235294
Precision  0.5283018867924528
Recall  0.9655172413793104
F1-score  0.6829268292682926
False Positive Rate  0.6410256410256411 /n
------Number of Clusters:  6 -----------
Accuracy  0.5441176470588235
Precision  0.4827586206896552
Recall  0.9655172413793104
F1-score  0.6436781609195403
False Positive Rate  0.7692307692307693 /n
------Number of Clusters:  7 ----

In [23]:
print("Accuracy Scores: ",cluster_accuracy_3)
print("Precision Scores: ",cluster_precision_3)
print("Recall Scores: ",cluster_recall_3)
print("F1 Scores: ",cluster_fscore_3)
print("False Positive Rates: ",cluster_fpr_3)

Accuracy Scores:  [0.6617647058823529, 0.7205882352941176, 0.7058823529411765, 0.6176470588235294, 0.5441176470588235, 0.6176470588235294, 0.6176470588235294, 0.6176470588235294]
Precision Scores:  [0.575, 0.6190476190476191, 0.6, 0.5283018867924528, 0.4827586206896552, 0.5306122448979592, 0.5294117647058824, 0.5272727272727272]
Recall Scores:  [0.7931034482758621, 0.896551724137931, 0.9310344827586207, 0.9655172413793104, 0.9655172413793104, 0.896551724137931, 0.9310344827586207, 1.0]
F1 Scores:  [0.6666666666666666, 0.732394366197183, 0.7297297297297297, 0.6829268292682926, 0.6436781609195403, 0.6666666666666666, 0.6749999999999999, 0.6904761904761904]
False Positive Rates:  [0.4358974358974359, 0.41025641025641024, 0.46153846153846156, 0.6410256410256411, 0.7692307692307693, 0.5897435897435898, 0.6153846153846154, 0.6666666666666666]
